# Analyse Hitzetage und Tropennächte

In [25]:
import requests 
import pandas as pd
import geopandas as gpd 
import matplotlib # for plotting
import folium
import branca

In [26]:
def get_stations() -> gpd.GeoDataFrame:
    response = requests.get(
            url=f"https://smart-urban-heat-map.ch/api/v2/latest"
    )
    stations = gpd.read_file(response.text)
    stations.drop(["temperature", "relativeHumidity", "dateObserved"], axis=1, inplace=True)
    
    return stations

def get_station_analysis(time_from: str, time_to: str) -> pd.DataFrame:
    stations = get_stations()
    
    stations['hitzetage'] = None
    stations['tropennaechte'] = None
    
    for idx, station in stations.iterrows():
        station_id = station.stationId
        response = requests.get(url=f"https://smart-urban-heat-map.ch/api/v2/timeseries?stationId={station_id}&timeFrom={time_from}&timeTo={time_to}")
        
        payload = response.json()
        if payload["values"] == []:
            continue
            
        df = pd.DataFrame(payload["values"])
    
        df["dateObserved"] = pd.to_datetime(df["dateObserved"])
        df["dateObserved"] = df["dateObserved"].dt.tz_convert("Europe/Zurich")
        
        # hier werden die Hitzetage und Tropennächte berechnet
        hitzetage = days_hotter_than(df, 30)
        tropennaechte = nights_hotter_than(df, 20)
        
        stations.at[idx, 'hitzetage'] = hitzetage
        stations.at[idx, 'tropennaechte'] = tropennaechte

    return stations

def days_hotter_than(df: pd.DataFrame, temperature: float) -> int:
    daily_maximums = df.groupby(
        df["dateObserved"].dt.date
    ).max()["temperature"]

    return (daily_maximums >= temperature).sum()

def nights_hotter_than(df: pd.DataFrame, temperature: float) -> int:
    df.loc[df['dateObserved'].dt.hour >= 18, 'nightDate'] = df["dateObserved"].dt.date + pd.Timedelta(days=1)
    df.loc[df['dateObserved'].dt.hour < 6, 'nightDate'] = df["dateObserved"].dt.date
        
    nightly_minimums = (
        df.groupby(df['nightDate'])['temperature']
       .min()
    )
        
    return (nightly_minimums > temperature).sum()

In [27]:
stations = get_stations()
stations

,stationId,name,geometry
0,11116,Ittigen Worblaufenstrasse Laterne,POINT (7.45978 46.97857)
1,11119,Zollikofen 3m,POINT (7.46407 46.99079)
2,11015,Viererfeld 2 (Schacht),POINT (7.43688 46.96271)
3,11120,Zollikofen Reichenbach Laterne,POINT (7.45139 46.99095)
4,11077,Morillongut (Schönegg) Mast,POINT (7.44007 46.93354)
...,...,...,...
96,11008,Kreuzung Brunnmatt-Schwarztorstrasse,POINT (7.42713 46.94479)
97,11023,"Obstberg, Wattenwylweg 32",POINT (7.46407 46.94714)
98,11042,Pärkli bei Eigerplatz,POINT (7.43094 46.94195)
99,11071,Steinhölzliwald,POINT (7.42805 46.93514)


In [28]:
# get data of June
time_from = "2024-05-31T22:00:00Z"
time_to = "2024-06-30T22:00:00Z"

station_analysis = get_station_analysis(time_from, time_to)
station_analysis

,stationId,name,geometry,hitzetage,tropennaechte
0,11044,Kasernenareal,POINT (7.45802 46.95696),2,0
1,11116,Ittigen Worblaufenstrasse Laterne,POINT (7.45978 46.97857),4,0
2,11119,Zollikofen 3m,POINT (7.46407 46.99079),1,0
3,11015,Viererfeld 2 (Schacht),POINT (7.43688 46.96271),3,0
4,11120,Zollikofen Reichenbach Laterne,POINT (7.45139 46.99095),1,0
...,...,...,...,...,...
95,11130,Bremgarten Ecke Lindenstrasse / Ritterstrasse ...,POINT (7.43882 46.97791),4,0
96,11008,Kreuzung Brunnmatt-Schwarztorstrasse,POINT (7.42713 46.94479),5,1
97,11023,"Obstberg, Wattenwylweg 32",POINT (7.46407 46.94714),4,0
98,11042,Pärkli bei Eigerplatz,POINT (7.43094 46.94195),4,0


In [29]:
# Hitzetage Map
m = folium.Map(location=[station_analysis.geometry.y.mean(), station_analysis.geometry.x.mean()], zoom_start=13, tiles="CartoDB positron")

colormap = branca.colormap.linear.YlOrRd_09
max_hitzetage = station_analysis.hitzetage.max()
colormap = colormap.scale(0, max_hitzetage).to_step(max_hitzetage) 
colormap.caption = "Anzahl Hitzetage"
colormap.add_to(m)

for idx, station in station_analysis.iterrows():
    folium.CircleMarker(
        location=(station.geometry.y, station.geometry.x),
        radius=5,
        color="black",
        weight=0.5,
        fill=True,
        fill_color=colormap(station.hitzetage),
        fill_opacity=1,
        tooltip=f"{station["name"]}: {station.hitzetage:.0f} Hitzetage",
    ).add_to(m)

m

In [30]:
# Tropennächte Map
m = folium.Map(location=[station_analysis.geometry.y.mean(), station_analysis.geometry.x.mean()], zoom_start=13, tiles="CartoDB positron")

colormap = branca.colormap.linear.YlOrRd_09
max_tropennaechte = station_analysis.tropennaechte.max() if station_analysis.tropennaechte.max() > 3 else 3
colormap = colormap.scale(0, max_tropennaechte).to_step(max_tropennaechte) 
colormap.caption = "Anzahl Tropennächte"
colormap.add_to(m)

for idx, station in station_analysis.iterrows():
    folium.CircleMarker(
        location=(station.geometry.y, station.geometry.x),
        radius=5,
        color="black",
        weight=0.5,
        fill=True,
        fill_color=colormap(station.tropennaechte),
        fill_opacity=1,
        tooltip=f"{station["name"]}: {station.tropennaechte:.0f} Tropennächte",
    ).add_to(m)

m